## Método de Elementos Finitos en dos dimensiones

El objetivo de este programa es resolver una ecuación diferencial lineal de segundo orden no homogénea en un intervalo acotado con el método de elementos finitos, el problema es el siguiente:

\begin{equation}
    -\Delta u+ u=f \quad \forall (x,y) \in R=[0,1] \times [0,1] : \partial R = g
\end{equation}

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import unit_square

In [2]:
def main(order = 1, maxh = 0.05):

    mesh = Mesh(unit_square.GenerateMesh(maxh=maxh)) # se genera una malla [0,1]x[0,1] especificando el tamaño de los elementos

    fes = H1(mesh, order= order, dirichlet="bottom|right|top|left") # se define el espacio utilizado para la solución

    u = fes.TrialFunction()  # symbolic object
    v = fes.TestFunction()   # symbolic object
    gfu = GridFunction(fes)  # solution
    
    # forma bilineal
    a = BilinearForm(fes, symmetric=True)
    a += (grad(u)*grad(v) + u*v)*dx
    a.Assemble()
    
    # forma bilineal
    f = LinearForm(fes)
    f += 1*v*dx
    f.Assemble()
    
    # función para la condición de Dirichlet
    g = 0
    gfu.Set(g, BND)
    
    # valores para resolver el sistema lineal considerando las condiciones de frontera
    r = f.vec.CreateVector()
    r.data = f.vec - a.mat * gfu.vec
    
    # solución del sistema
    gfu.vec.data += a.mat.Inverse(freedofs=fes.FreeDofs()) * r
    
    return gfu, mesh

gfu, mesh = main()

Draw(gfu)

WebGuiWidget(value={'ngsolve_version': '6.2.2104', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

BaseWebGuiScene